## SALSA Clump UV Background Clump-Labeling Issues

In [1]:
# importing necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import pickle
import configparser
import sys
import os
from mpi4py import MPI
import yt
import trident
import salsa
from salsa.utils import check_rays
import scipy.integrate as sp

/opt/software/Python/3.6.4-foss-2018a/lib/python3.6/_collections_abc.py:666: MatplotlibDeprecationWarning: The global colormaps dictionary is no longer considered public API.
  self[key]


### Running SALSA Pipeline

Here, I give a breif overview as to how we obtain our SALSA data using pre-generated ray data.

To begin with, we start by loading in the halo data from Foggie that we would be using

In [2]:
halo_path = "some file path"

halo_data = yt.load(halo_path)

yt : [ERROR    ] 2023-09-07 12:37:12,710 None of the arguments provided to load() is a valid file
yt : [ERROR    ] 2023-09-07 12:37:12,711 Please check that you have used a correct path


YTOutputNotIdentified: Supplied ('some file path',) {}, but could not load!

Now I create the rays that we will be using

In [ ]:
center = halo_data.arr[,'kpc']

nrays = 1

max_impact = 

gal_vel = 

other_fields = [("density","g/cm**3"), "temperature", ("metallicity","Zsun"), "radius"]

out_path = "./"


salsa.generate_lrays(halo_data, 
                     center.to('code_length'), 
                     nrays, 
                     max_impact, 
                     length=600, 
                     field_parameters={'bulk_velocity':gal_vel}, 
                     ion_list=['H I'], 
                     fields=other_fields, 
                     out_dir=out_path)

Once the dataset has been loaded in, and the rays have been created, we can set our desired UV Background. In this code, I use the UV Background from Puchwein et al. 2019, but the same process can be followed with the other UVB that is analyzed from Haart & Madau 2012. All that needs to be changed is the respective file path.

In [ ]:
ray = 0
ion = "O VI"
atom,istate = ion.split(" ")

field_name = f"{atom}_p{trident.from_roman(istate)-1}_number_density"

abuns = dict(zip(elements, abundances[1]))

# uvb = "/mnt/home/tairaeli/trident_uncertainty/mods/abundances/data_bin/hm2012_ss_hr.h5"
uvb = "/mnt/home/tairaeli/trident_uncertainty/mods/abundances/data_bin/par_test.h5"
# uvb_name = "HM_2012"
uvb_name = "PCW_2019"

ray_dat = yt.load(f"/mnt/scratch/tairaeli/halo2392_pcw_2019/redshift2.0/rays/ray{ray}.h5")

trident.add_ion_number_density_field(atom, trident.from_roman(istate), 
                                     ray_dat, abundance_dict = abuns, 
                                     ionization_table = uvb)

pcw_dens = ray_dat.r[("gas",field_name)].copy()

ray_len_cm = ray_dat.domain_width.to("cm")[0]
n_cells = len(dens)

ray_pos = np.linspace(0,ray_len_cm,n_cells)

For the sake of simplicity (as well as avoiding a strange quirk with yt when it comes to setting UVBs), the results of the Haart and Madau UVB has already been saved to a pickle file.

In [ ]:
with open("./hm_dens.pickle", "rb") as dens_dat:
    hm_dens = pickle.load(dens_dat)

Now, given some abudnance table, we use SALSA to identify where the gas clumps are within our ray for each UVB (also showing this process for only the Puchwein UVB)

In [ ]:
abundances = abun.iloc[0].to_dict()
            
abs_ext = salsa.AbsorberExtractor(ds, 
                                  ray_file, 
                                  ion_name = ion, 
                                  velocity_res =20, 
                                  abundance_table = abundances, 
                                  calc_missing=True)

pcw_clump_dat = salsa.get_absorbers(abs_ext, 
                         my_rays, 
                         method='spice', 
                         fields=other_fields, 
                         units_dict=field_units)

pcw_clump_dat = pcw_clump_dat.drop(columns='index')

Once again, the output data for both UV Backgrounds have been saved to pickle files

In [ ]:
with open("./salsa_out_hm", "rb") as salsa_dat:
    hm_clump_dat = pickle.load(salsa_dat)

### Displaying Results

Now, we create a plot of the densities for a particular ion as a function of the position along the ray. We also fill in regions where SALSA flagged a region along the ray as a "clump".

In [ ]:
colors = plt.cm.cool([0.3,0.4,0.9,1])

plt.figure(figsize = [15,8], dpi = 500, facecolor = "white")
plt.semilogy(ray_pos, hm_dens,label = "HM 2012", color = colors[3])
plt.semilogy(ray_pos, pcw_dens,label = "PCW 2019", color = colors[1])
# plt.axhline(1e-13)

hm_clumps = hm_clump_dat[ion]["HM_2012"][hm_dat[ion]["HM_2012"]["lightray_index"] == str(ray)]
pcw_clumps = pcw_clump_dat[ion]["PCW_2019"][pcw_dat[ion]["PCW_2019"]["lightray_index"] == str(ray)]

i = 0
for i in range(len(hm_clumps["interval_start"])):
    
    lb = hm_clumps["interval_start"][i]
    hb = hm_clumps["interval_end"][i]
    
    rng = [lb,hb]
    
    yb = hm_dens[slice(*rng)]
    xb = np.arange(*rng)
    
    plt.fill_between(xb,yb, color = colors[2], alpha = 0.3)

for i in range(len(pcw_clumps["interval_start"])):
    
    lb = pcw_clumps["interval_start"][i]
    hb = pcw_clumps["interval_end"][i]
    
    rng = [lb,hb]
    
    yb = dens[slice(*rng)]
    xb = np.arange(*rng)
    
    plt.fill_between(xb,yb, color = colors[0], alpha = 0.3)    

plt.grid()
plt.legend(fontsize = 20)
# plt.ylim(10**(-15),10**(-6.5))
# plt.xlim(500,900)
plt.xlabel("Position Along Ray", fontsize = 25)
plt.ylabel(r"Density ($cm^{-3}$)", fontsize = 25)
plt.title(f"UVB Number Density Comparison", fontsize = 30)

Here, the purple regions represent where SALSA found a clump in both UVBs and the pink and blue regions represent where SALSA only recognized a clump in 1 of these regions

### Highlighting Issues